In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings

In [9]:
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

In [34]:
#키워드 검색
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

# 키워드를 검색한 유튜브 주소 접속
keyword = '뷔스티에 원피스'

url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
driver.get(url)

driver.maximize_window()

# 필터 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/tp-yt-paper-button').click()
time.sleep(1)

# 구분 -> 동영상 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[1]/a/div/yt-formatted-string').click()

# 업로드 날짜 -> 올해 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a').click()

In [35]:
# 제목 & 링크 크롤링

# 스크롤 다운 10번 실행
body = driver.find_element_by_tag_name('body')
body.send_keys(Keys.PAGE_DOWN)

for i in range(1,3):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)

soup = BeautifulSoup(driver.page_source, 'html.parser')
name = soup.select('a#video-title')
video_url = soup.select('a#video-title')

video_num_list = [] #영상번호
name_list = [] #영상제목
url_list = [] #영상주소

for i in range(len(name)):
    video_num_list.append(i+1)
    name_list.append(name[i].text.strip())

for i in video_url:
    url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))


In [20]:
#업로드일 가져오기
day_list = [] #업로드일

#업로드 일자 가져오기
for i in range(0,len(name_list)):
    if url_list[i].split('/')[3] == 'shorts':  #쇼츠

        print(i, ': 쇼츠')
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        driver.get(url_list[i])

        time.sleep(3)

        #더보기 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
        time.sleep(2)

        #설명 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
        time.sleep(4)

        #업로드일 정보 가져오기
        day = driver.find_element_by_id("publish-time").text.split(':')[1].strip()

        print(day)
        day = day.replace('. ', '-')
        day_list.append(day)
        time.sleep(3)

    else :
        print(i, ': 일반')

        driver.get(url_list[i])
            
        time.sleep(3)    

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        time.sleep(3)  

        #업로드일 정보 가져오기
        #day = soup.select('#formatted-snippet-text > span:nth-child(3)')[0].text
        day = driver.find_element_by_id("info-strings").text.split('\n')[0]
        
        print(day)
        day = day.replace('. ', '-')
        day_list.append(day)
        time.sleep(3)

0 : 일반
2021. 11. 19.
1 : 일반
2021. 7. 29.
2 : 일반
2022. 4. 5.
3 : 일반
2021. 8. 7.


In [31]:
youtubeDic = {
    '제목': name_list,
    '업로드일': day_list,
    '주소': url_list,
}

youtubeDf = pd.DataFrame(youtubeDic)
youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])
youtubeDf.head()

,제목,업로드일,주소
0,150 / 160 / 170cm 다양한 소재의 원피스 핏 보여드려요👗 (+원피스코디...,2021-11-19,https://www.youtube.com/watch?v=KYJj06Ly9Ao
1,"2172978)뷔스티에 롱 원피스/패턴,Bustier Long Dress",2021-07-29,https://www.youtube.com/watch?v=I7UOVYz9C0M
2,뷔스티에원피스 추천 판매순위 Top10 || 가격 평점 후기 비교,2022-04-05,https://www.youtube.com/watch?v=0z7UMFSWyN8
3,여름엔 원피스 하나로 충분해! 베스트 중 베스트만 모았다💚 77사이즈 통통녀 여름 ...,2021-08-07,https://www.youtube.com/watch?v=oZr5DTD4fM0


In [24]:
youtubeDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   영상 번호   4 non-null      int64         
 1   제목      4 non-null      object        
 2   업로드일    4 non-null      datetime64[ns]
 3   주소      4 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 256.0+ bytes


In [32]:
youtubeDf.to_csv('C:\jupiter_workspace\zam_project\outputs/youtube_'+keyword +'.csv', encoding='utf-8-sig', index=False)